#### Compare Biolume Proxy Calculations loaded from stoqs_auv_compare Parquet output

This Notebook is part of the auv-python project (private repository at https://github.com/mbari-org/auv-python). It demonstrates how to read and make interactive plots of millions of data points accessed from a STOQS database.

To execute it (for example):

```bash
    cd GitHub  # Or other appropriate directory on your computer
    git clone https://github.com/mbari-org/auv-python.git
    cd auv-python
    poetry install
    poetry shell
    cd notebooks
    jupyter notebook
    # Open this notebook and run it in Chrome - interactive zooming does not work in VS Code
```

The urls in the pooch.retrieve() calls  below were generated by going to https://stoqs.shore.mbari.org/stoqs_auv_compare/ and clicking the buttons of the Measured Parameters to be included in the Parquet file. Then clicking the "Measured Parameter Data Access" section and clicking the "Estimate requirements" button to verify that the estimated values are within the available values of the server. 

In [ ]:
# Do all the imports here and then load the data so that we can randomly execute
# any of the plotting cells below

import colorcet
import holoviews as hv
import hvplot.pandas
import os
import ipywidgets as widgets
import pandas as pd
import panel as pn
import pooch
import statsmodels.api as sm
from bokeh.models.formatters import PrintfTickFormatter
from holoviews.operation.datashader import datashade
hv.extension("bokeh")

In [ ]:
# Takes several minutes to retrieve the data the first time, thereafter it's read from a local cache
matlab_proxies = pooch.retrieve(
    url="https://stoqs.shore.mbari.org/stoqs_auv_compare/api/measuredparameter.parquet?parameter__name=adinos&parameter__name=bg_biolum+%28ph+L%5E%7B-1%7D%29&parameter__name=diatoms&parameter__name=hdinos&parameter__name=intflash+%28ph+s%5E%7B-1%7D%29&parameter__name=nbflash_high+%28L%5E%7B-1%7D%29&parameter__name=nbflash_low+%28L%5E%7B-1%7D%29&parameter__name=profile&collect=name&include=activity__name",
    known_hash="e718db2ce310e2ac5dd3a7f69d50fa418942989ea81ca9c7c3bf21d7c4417090",
)
dfm = pd.read_parquet(matlab_proxies)
dfm.describe()

In [ ]:
dfm

In [ ]:
# Takes several minutes to retrieve the data the first time, thereafter it's read from a local cache
python_proxies = pooch.retrieve(
    url="https://stoqs.shore.mbari.org/stoqs_auv_compare_t/api/measuredparameter.parquet?parameter__name=biolume_bg_biolume+%28photons%2Fliter%29&parameter__name=biolume_intflash+%28photons%2Fs%29&parameter__name=biolume_nbflash_high+%28flashes%2Fliter%29&parameter__name=biolume_nbflash_low+%28flashes%2Fliter%29&parameter__name=biolume_proxy_adinos&parameter__name=biolume_proxy_diatoms&parameter__name=biolume_proxy_hdinos&parameter__name=profile_number&collect=name&include=activity__name",
    known_hash="a0a6b5ba991e56d6428e0e4920eb83730a31a63007043f8ca490566678250bc0",
)
dfp = pd.read_parquet(python_proxies)
dfp.describe()

In [ ]:
dfp.loc[['dorado']]  # Don't show the Gulper Activities

In [ ]:
# The following cells make time series comparison plots all of the diamond mission data, in order:
# 'adinos', 'bg_biolum', 'diatoms', 'hdinos', 'intflash', 'nbflash_high', 'nbflash_low', and 'profile'
# Do not commit following cell outputs to the repository - they are too big!

non_time_indx = ['platform', 'activity__name', 'depth', 'latitude', 'longitude']

In [ ]:
# Merge the Matlab and Python dataframes for making comparison biplots
if os.path.exists("merged_diamond_missions.parquet"):
    print("Reading a saved copy of the merged dataframes")
    df = pd.read_parquet("merged_diamond_missions.parquet")
else:
    print("Merging the dataframes is necessary to make the plots - this can take several minutes")
    df = pd.merge(
        dfm.droplevel(non_time_indx).resample("2S").mean(),
        dfp.droplevel(non_time_indx),
        how="inner",
        left_index=True,
        right_index=True,
    )
    df.to_parquet("merged_diamond_missions.parquet")
df.head()

In [ ]:
# TODO: Make this widget work - See https://github.com/mbari-org/auv-python/issues/1
platform = pn.widgets.Select(options=['dorado', 'dorado_Gulper'], name='Platform')
#dfpd = dfp['biolume_intflash (photons/s)'].reset_index(['depth']).droplevel(['platform', 'latitude', 'longitude'])
#dfpd = dfp.reset_index(['depth']).droplevel(['platform', 'latitude', 'longitude'])

def get_dfpd(platform):
    # Make depth a column we can filter on and filter on selected platform
    df = dfp.reset_index(['platform', 'depth'])[platform].droplevel(['latitude', 'longitude'])
    return df

'''
dfpdi = hvplot.bind(get_dfpd, platform).interactive()

depth_range = pn.widgets.RangeSlider(start=dfpdi.depth.min(), end=dfpdi.depth.max(), value=(dfpdi.depth.min(), dfpdi.depth.max()), format=PrintfTickFormatter(format='%.1f m'))
#dfpd[(dfpd.depth>=depth_range.value_start) & (dfpd.depth<=depth_range.value_end)]
dfpdi[(dfpdi.depth>=depth_range.value_start) & (dfpdi.depth<=depth_range.value_end)].hvplot(grid=True)
'''

In [ ]:
# Pick variable for time series plot
vars = {"adinos": "biolume_proxy_adinos",
        "bg_biolum (ph L^{-1})": "biolume_bg_biolume (photons/liter)",
        "diatoms": "biolume_proxy_diatoms",
        "hdinos": "biolume_proxy_hdinos",
        "intflash (ph s^{-1})": "biolume_intflash (photons/s)",
        "nbflash_high (L^{-1})": "biolume_nbflash_high (flashes/liter)",
        "nbflash_low (L^{-1})": "biolume_nbflash_low (flashes/liter)",
        "profile": "profile_number",
        }
var_picked = widgets.Dropdown(options=vars.keys(), name='Variable')
display(var_picked)

In [ ]:
m_plot = dfm.droplevel(non_time_indx)[var_picked.value].hvplot(width=800, height=300)
p_plot = dfp.droplevel(non_time_indx)[vars[var_picked.value]].hvplot()
m_plot * p_plot

The following cells make comparison biplots of all of the diamond mission data, in order:
'adinos', 'bg_biolum', 'diatoms', 'hdinos', 'intflash', 'nbflash_high', 'nbflash_low', and 'profile'
There should be a slope of 1.0 for all of the plots

In [ ]:
def biplot(df, x="adinos", y="biolume_proxy_adinos", resample_mean: str='2S', rolling_mean: int=0):
    # Use statsmodels and datashader to print regression info and make a biplot
    if rolling_mean:
        dfa = df[[x, y]].dropna().rolling(rolling_mean).mean().dropna()
    else:
        dfa = df[[x, y]].dropna().resample(resample_mean).mean().dropna()
    results = sm.OLS(dfa[y], dfa[x]).fit()
    print(results.summary())
    slope_plot = hv.Slope.from_scatter(hv.Scatter(dfa.to_numpy())).opts(line_width=1, color='red')
    pts = hv.Points(dfa, [x, y])
    # { and } cause problems in opts title, so we need to replace them in the x variables
    title = y + " = " + f"{results.params[0]:.4f}" + " * " + x.replace("{", r"").replace("}", "")
    return datashade(pts, cmap=colorcet.linear_blue_5_95_c73).opts(width=700, height=700, aspect='equal', title=title) * slope_plot

In [ ]:
biplot(df, x="adinos", y="biolume_proxy_adinos")

In [ ]:
biplot(df, x="bg_biolum (ph L^{-1})", y="biolume_bg_biolume (photons/liter)")

In [ ]:
biplot(df, x="bg_biolum (ph L^{-1})", y="biolume_bg_biolume (photons/liter)")

In [ ]:
biplot(df, x="diatoms", y="biolume_proxy_diatoms")

In [ ]:
biplot(df, x="hdinos", y="biolume_proxy_hdinos")

In [ ]:
biplot(df, x="intflash (ph s^{-1})", y="biolume_intflash (photons/s)")

In [ ]:
biplot(df, x="nbflash_high (L^{-1})", y="biolume_nbflash_high (flashes/liter)")

In [ ]:
biplot(df, x="nbflash_low (L^{-1})", y="biolume_nbflash_low (flashes/liter)")

In [ ]:
biplot(df, x="profile", y="profile_number")

The `nbflash_high` and `nbflash_low` plots might be improved with some filtering to smooth out some of the integerized nature of the count data.

In [ ]:
biplot(df, x="nbflash_high (L^{-1})", y="biolume_nbflash_high (flashes/liter)", resample_mean="60S")

In [ ]:
biplot(df, x="nbflash_high (L^{-1})", y="biolume_nbflash_high (flashes/liter)", rolling_mean=30)

In [ ]:
biplot(df, x="nbflash_low (L^{-1})", y="biolume_nbflash_low (flashes/liter)", resample_mean="60S")

In [ ]:
biplot(df, x="nbflash_low (L^{-1})", y="biolume_nbflash_low (flashes/liter)", rolling_mean=30)